In [1]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense

from keras import backend as K
img_dim_ordering = 'tf'
K.set_image_dim_ordering(img_dim_ordering)

import numpy as np

Using TensorFlow backend.


In [2]:
from util import *

In [3]:
data_dir = 'data'

In [4]:
X_train_imgs_filename = '%s/X_train_imgs.csv' % data_dir
X_val_imgs_filename = '%s/X_val_imgs.csv' % data_dir
X_test_imgs_filename = '%s/X_test_imgs.csv' % data_dir
y_train_filename = '%s/y_train.csv' % data_dir
y_val_filename = '%s/y_val.csv' % data_dir
y_test_filename = '%s/y_test.csv' % data_dir

In [5]:
X_train_img_files = read_img_file_csv(X_train_imgs_filename)
X_val_img_files = read_img_file_csv(X_val_imgs_filename)
X_test_img_files = read_img_file_csv(X_test_imgs_filename)
y_train = read_label_file_csv(y_train_filename)
y_val = read_label_file_csv(y_val_filename)
y_test = read_label_file_csv(y_test_filename)

In [6]:
def pretrained_model(img_shape, num_classes, activation='relu', n=4096):
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
    #model_vgg16_conv.summary()
    
    #Create your own input format
    keras_input = Input(shape=img_shape, name = 'image_input')
    
    #Use the generated model 
    output_vgg16_conv = model_vgg16_conv(keras_input)
    
    #Add the fully-connected layers 
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(n, activation=activation, name='fc1')(x)
    x = Dense(n, activation=activation, name='fc2')(x)
    x = Dense(num_classes, activation='softmax', name='predictions')(x)
    
    #Create your own model 
    pretrained_model = Model(inputs=keras_input, outputs=x)
    pretrained_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return pretrained_model

In [7]:
X_train_imgs = read_imgs(X_train_img_files, scale_to_255=True)
X_train_imgs.shape

(12432, 64, 64, 3)

In [8]:
X_train_img_files[:5]

['data/non-vehicles/GTI/image1314.png',
 'data/non-vehicles/GTI/image1630.png',
 'data/non-vehicles/GTI/image1838.png',
 'data/non-vehicles/GTI/image3168.png',
 'data/vehicles/KITTI_extracted/3822.png']

In [9]:
def vgg_load_imgs(X_img_files):
    X_img_arr = []
    for img_path in tqdm(X_img_files):
        #print(img_path)
        curr_img = image.load_img(img_path, target_size=(64, 64))
        X_img_arr.append(curr_img.copy())
        curr_img.close()

    X_imgs = np.stack(X_img_arr)    
    
    return X_imgs

In [10]:
X_train_img_files, y_train = shuffle_train_data(X_train_img_files, y_train)

In [11]:
cutoff = int(0.1*len(y_train))
_X_train_img_files = X_train_img_files[:cutoff]
_y_train = y_train[:cutoff]

In [12]:
_X_train = vgg_load_imgs(_X_train_img_files)
_X_train.shape

100%|██████████| 1243/1243 [00:04<00:00, 288.72it/s]


(1243, 64, 64, 3)

In [13]:
X_val = vgg_load_imgs(X_val_img_files)
X_val.shape

100%|██████████| 3569/3569 [00:21<00:00, 167.21it/s]


(3569, 64, 64, 3)

In [ ]:
model = pretrained_model((64, 64, 3), 2, n=256)

In [ ]:
model.fit(x=_X_train, y=np.array(_y_train), batch_size=32, epochs=10, verbose=1, callbacks=None, 
          validation_data=(X_val, np.array(y_val)), shuffle=True, initial_epoch=0)

Train on 1243 samples, validate on 3569 samples
Epoch 1/10
1243/1243 [==============================] - 722s 581ms/step - loss: 8.3889 - acc: 0.4739 - val_loss: 7.8445 - val_acc: 0.5133